In [2]:
import numpy as np
import matplotlib.pyplot as plt
import fid_custom
import os

In [3]:
celebA_data = 'celebA_data/celebA_fid_stats.npz'
biased_data = 'biased_datasets/Smiling_Young_Male_99_Not_Male_1_N15000/fid_stats.npz'

real_stats = fid_custom.load_stats(celebA_data)
biased_stats = fid_custom.load_stats(biased_data)

mu_r, sigma_r, Nr = list(real_stats.values())
mu_g, sigma_g, Ng = list(biased_stats.values())

reference = fid_custom.calculate_fid(mu_r, sigma_r, mu_g, sigma_g)

In [4]:
reference

np.float64(34.47328366822464)

In [36]:
folder_path = 'GANs/features/gan_fid_v4_exact01_10000'

data_list = []
for file in os.listdir(folder_path):
    if file.endswith(".npy"):
        file_path = os.path.join(folder_path, file)
        data_list.append(np.load(file_path))

In [37]:
X = np.array(data_list)
X.shape

(5000, 2048)

In [38]:
def mu_sigma_update(mu, sigma, N, f):
    diff = f - mu
    mu_new = mu + diff/(N + 1)
    sigma_new = ((N - 1) * sigma) / N + np.outer(diff, diff) / (N + 1)
    
    return mu_new, sigma_new, N+1

In [39]:
def mu_sigma_update_batch(mu_a, sigma_a, Na, mu_b, sigma_b, Nb):
    mu_new = (Na*mu_a + Nb*mu_b)/ (Na + Nb)
    sigma_new = ((Na-1)*sigma_a + (Nb-1)*sigma_b + Na*Nb*np.outer(mu_a-mu_b, mu_a-mu_b)/(Na+Nb))/(Na+Nb-1)
    return mu_new, sigma_new, Na+Nb

In [40]:
mu, sigma, N = mu_g, sigma_g, Ng
fid_list = [reference]
b = 10

In [41]:
for i in range(0,5000, b):
    print(f"Processing {i}/5000, curr_num = {N}", end="\r", flush=True)
    x_batch = X[i:i+b]
    mu_b = np.mean(x_batch, axis=0)
    sigma_b = np.cov(x_batch, rowvar=False)
    
    mu, sigma, N = mu_sigma_update_batch(mu, sigma, N, mu_b, sigma_b, b)
    fid_list.append(fid_custom.calculate_fid(mu_r, sigma_r, mu, sigma))
    

In [42]:
np.save("fid_list_data/data_gan_fid_lossfv4_exact01.npy", fid_list)